#### Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [2]:
res = requests.get('https://habr.com/ru/')
res

<Response [200]>

In [3]:
req = requests.get('https://habr.com/ru/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
posts = soup.find_all('article', class_='post')
len(posts)

20

In [5]:
import re

In [6]:
keywords = ['Raspberry ', 'Гаджеты', 'python', 'JavaScript', 'Node.JS', 'Управление проектами', 'alkaline']

In [7]:
my_posts = {}
my_keywords = []
for post in posts:
    for keyword in keywords:
        if re.findall(keyword, post.text):
            my_keywords.append(keyword)
            post_id = post.parent.attrs.get('id')
            post_id = int(post_id.split('_')[-1])
            my_posts.setdefault(post_id, my_keywords)    
    my_keywords = []     
print(my_posts)

{510900: ['JavaScript'], 511522: ['JavaScript', 'Node.JS'], 511470: ['JavaScript']}


In [8]:
my_posts = {}
my_keywords = []
for post in posts:
    for keyword in keywords:
        if re.findall(keyword, post.text):
            my_keywords.append(keyword)
            post_link = post.find('a', class_='post__title_link').get('href')
            my_posts.setdefault(post_link, my_keywords)    
    my_keywords = []     
print(my_posts)

{'https://habr.com/ru/company/ruvds/blog/510900/': ['JavaScript'], 'https://habr.com/ru/post/511522/': ['JavaScript', 'Node.JS'], 'https://habr.com/ru/post/511470/': ['JavaScript']}


In [9]:
results = []
result = []
for post in posts:
    for keyword in keywords:
        if re.findall(keyword, post.text):
            post_date = post.find('span', class_='post__time').get_text()
            post_header = post.find('a', class_='post__title_link').get_text()
            post_link = post.find('a', class_='post__title_link').get('href')
            result = [post_date, post_header, post_link]
            if result not in results: # проверка, что пост уже был добавлен
                results.append(result)
print(results)

[['вчера в 16:16', '4 революционных возможности JavaScript из будущего', 'https://habr.com/ru/company/ruvds/blog/510900/'], ['вчера в 12:47', 'Node.js + MongoDB: перформанс транзакций', 'https://habr.com/ru/post/511522/'], ['вчера в 14:28', 'Inertia.js – современный монолит', 'https://habr.com/ru/post/511470/']]


In [10]:
df = pd.DataFrame(results, columns=['Дата', 'Заголовок', 'Ссылка'])
df

,Дата,Заголовок,Ссылка
0,вчера в 16:16,4 революционных возможности JavaScript из буду...,https://habr.com/ru/company/ruvds/blog/510900/
1,вчера в 12:47,Node.js + MongoDB: перформанс транзакций,https://habr.com/ru/post/511522/
2,вчера в 14:28,Inertia.js – современный монолит,https://habr.com/ru/post/511470/


#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

#### Алгоритм
1. Нахожу ссылку на каждый пост на главной странице (всего 20)

2. Открываю каждую и ищу вхождения из keywords. 

3. Если нашёл, то добавляю в result.

4. Результирующий список results.append(results)

In [11]:
keywords = ['Софт', 'Физика']

In [12]:
results = []
result = []
for post in posts:
    post_link = post.find('a', class_='post__title_link').get('href')
    post_content = BeautifulSoup(requests.get(post_link).text, 'html.parser')
    for keyword in keywords:
        if re.findall(keyword, post_content.text):
            post_date = post_content.find('span', class_='post__time').get_text()
            post_header = post_content.find('span', class_='post__title-text').get_text()
            post_text = post_content.find('div', class_='post__text').get_text()
            result = [post_date, post_header, post_link, post_text]
            results.append(result)
len(results)

# post_tag.text

0

In [13]:
df = pd.DataFrame(results, columns=['Дата', 'Заголовок', 'Ссылка', 'Текст статьи'])
df

,Дата,Заголовок,Ссылка,Текст статьи


### Вопрос
Почему первая строчка с текстом статьи начинается на \n\r\n и как это можно обойти?

#### Задание 2.

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

In [14]:
params = {
    "email": "sd@mail.ru"
}
req = requests.post('https://digibody.avast.com/v1/web/leaks', json=params)
req.json()

{'status': 'ok',
 'value': [{'leak_id': '417a54c5-89b5-4ea5-93ae-a56744c32b7b',
   'username': 'ishay',
   'domain': 'livejournal.com',
   'passwords': [{'password': None, 'encrypted': False}],
   'leak_info': {'id': '417a54c5-89b5-4ea5-93ae-a56744c32b7b',
    'date': 1558569600000,
    'title': 'LiveJournal',
    'description': "In 2017, social network LiveJournal's database was allegedly breached. The stolen data contains passwords, email addresses and additional personal information. This breach is being privately shared in the dark web.",
    'number_of_entries': 30485836,
    'source_references': [],
    'media_references': [],
    'domains': ['livejournal.com'],
    'picture_url': 'https://pamcdn.avast.com/pamcdn/icons/livejournal_com.png',
    'service_name': 'LiveJournal'},
   'marked_resolved': False,
   'leak_date': 1558569600000,
   'passwords_count': 1},
  {'leak_id': '417a54c5-89b5-4ea5-93ae-a56744c32b7b',
   'username': 'sd_ds',
   'domain': 'livejournal.com',
   'passwor

In [15]:
df = pd.DataFrame(req.json()['value'])
df

,leak_id,username,domain,passwords,leak_info,marked_resolved,leak_date,passwords_count
0,417a54c5-89b5-4ea5-93ae-a56744c32b7b,ishay,livejournal.com,"[{'password': None, 'encrypted': False}]","{'id': '417a54c5-89b5-4ea5-93ae-a56744c32b7b',...",False,1558569600000,1
1,417a54c5-89b5-4ea5-93ae-a56744c32b7b,sd_ds,livejournal.com,"[{'password': None, 'encrypted': False}]","{'id': '417a54c5-89b5-4ea5-93ae-a56744c32b7b',...",False,1558569600000,1
2,cf7afbbb-7695-4f97-b755-3d63df87828f,sd@mail.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'cf7afbbb-7695-4f97-b755-3d63df87828f',...",False,1549411200000,1
3,f9afdb14-e6f7-4c96-a7ed-457e95c5442a,sd@mail.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'f9afdb14-e6f7-4c96-a7ed-457e95c5442a',...",False,1593043200000,1
4,bf72e46c-14f7-4990-ad45-08530802c09f,lawrik,cityofvillains.com,"[{'password': None, 'encrypted': False}]","{'id': 'bf72e46c-14f7-4990-ad45-08530802c09f',...",False,1580947200000,1
5,5fad9982-2c7d-4a3e-8209-fa1c8ae76c07,sd7506,canva.com,"[{'password': None, 'encrypted': True}]","{'id': '5fad9982-2c7d-4a3e-8209-fa1c8ae76c07',...",False,1560384000000,1
6,b14f73ee-e3be-4152-bf52-069a837c7589,lawrik,wowprogress.com,"[{'password': None, 'encrypted': False}]","{'id': 'b14f73ee-e3be-4152-bf52-069a837c7589',...",False,1575504000000,1
7,71db3f29-006d-4158-a3f0-8102652fbf57,sd@mail.ru,,"[{'password': None, 'encrypted': False}]","{'id': '71db3f29-006d-4158-a3f0-8102652fbf57',...",False,1549411200000,1
8,531f3eba-8ffc-4fc1-ba14-a24db354997b,azofix,ozonemc.ru,"[{'password': None, 'encrypted': True}]","{'id': '531f3eba-8ffc-4fc1-ba14-a24db354997b',...",False,1570665600000,1
9,a62f898e-dc99-4ff7-b28d-1247a9a33e5e,rm_ewoqqqq,,"[{'password': None, 'encrypted': True}]","{'id': 'a62f898e-dc99-4ff7-b28d-1247a9a33e5e',...",False,1581552000000,1


In [16]:
emails = ['aekomin@gmail.com']

In [17]:
result = {}
results = {}
for email in emails:
    params = {
    "email": email
}
    req = requests.post('https://digibody.avast.com/v1/web/leaks', json=params)
    result = req.json()
    results.update(result)

In [18]:
df = pd.DataFrame(results['value'])
df

,leak_id,username,domain,passwords,leak_info,marked_resolved,leak_date,passwords_count
0,037a277e-d484-4c34-8473-904024e489e1,aekomin@gmail.com,,"[{'password': None, 'encrypted': False}]","{'id': '037a277e-d484-4c34-8473-904024e489e1',...",False,1513900800000,1
1,f77795b3-f443-426c-90fb-44bb13211cfc,aekomin@gmail.com,,"[{'password': None, 'encrypted': False}]","{'id': 'f77795b3-f443-426c-90fb-44bb13211cfc',...",False,1545350400000,1
2,68a35a62-3f01-45ea-a345-b2d56ef539f2,cooltouch,android.net,"[{'password': None, 'encrypted': True}]","{'id': '68a35a62-3f01-45ea-a345-b2d56ef539f2',...",False,1565222400000,1
3,8cabbe89-a11e-4cd9-9757-63efc704b6f8,589400354,myspace.com,"[{'password': None, 'encrypted': True}]","{'id': '8cabbe89-a11e-4cd9-9757-63efc704b6f8',...",False,1477008000000,1


В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [19]:
result = {}
results ={}
for email in emails:
    params = {
    "email": email
}
    req = requests.post('https://digibody.avast.com/v1/web/leaks', json=params)
    result = req.json()
    for item in result['value']: ### добавляю искомый email в словарь
        item['e-mail'] = email 
    results.update(result)

In [20]:
df = pd.DataFrame(results['value'])
df

,leak_id,username,domain,passwords,leak_info,marked_resolved,leak_date,passwords_count,e-mail
0,037a277e-d484-4c34-8473-904024e489e1,aekomin@gmail.com,,"[{'password': None, 'encrypted': False}]","{'id': '037a277e-d484-4c34-8473-904024e489e1',...",False,1513900800000,1,aekomin@gmail.com
1,f77795b3-f443-426c-90fb-44bb13211cfc,aekomin@gmail.com,,"[{'password': None, 'encrypted': False}]","{'id': 'f77795b3-f443-426c-90fb-44bb13211cfc',...",False,1545350400000,1,aekomin@gmail.com
2,68a35a62-3f01-45ea-a345-b2d56ef539f2,cooltouch,android.net,"[{'password': None, 'encrypted': True}]","{'id': '68a35a62-3f01-45ea-a345-b2d56ef539f2',...",False,1565222400000,1,aekomin@gmail.com
3,8cabbe89-a11e-4cd9-9757-63efc704b6f8,589400354,myspace.com,"[{'password': None, 'encrypted': True}]","{'id': '8cabbe89-a11e-4cd9-9757-63efc704b6f8',...",False,1477008000000,1,aekomin@gmail.com


In [21]:
col_list = df.columns.tolist()
col_list

['leak_id',
 'username',
 'domain',
 'passwords',
 'leak_info',
 'marked_resolved',
 'leak_date',
 'passwords_count',
 'e-mail']

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [22]:
df = df[['e-mail', 'leak_date', 'domain', 'leak_info']]
df

,e-mail,leak_date,domain,leak_info
0,aekomin@gmail.com,1513900800000,,"{'id': '037a277e-d484-4c34-8473-904024e489e1',..."
1,aekomin@gmail.com,1545350400000,,"{'id': 'f77795b3-f443-426c-90fb-44bb13211cfc',..."
2,aekomin@gmail.com,1565222400000,android.net,"{'id': '68a35a62-3f01-45ea-a345-b2d56ef539f2',..."
3,aekomin@gmail.com,1477008000000,myspace.com,"{'id': '8cabbe89-a11e-4cd9-9757-63efc704b6f8',..."


#### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.

Документация к API VK: https://vk.com/dev/methods

GROUP = 'netology’

TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [51]:
url = 'https://api.vk.com/method/wall.get?'
token = 'e516bbfde516bbfde516bbfd52e565a2cfee516e516bbfdba02b1c943e606e87c4b74c4'
version = '5.103'
sleep = 0.33

In [52]:
params = {
    'access_token': token,
    'domain': 'netology',
    'v': version,
    'count': 50
}

In [53]:
r = requests.get(url, params)

In [54]:
r.text

'{"response":{"count":7095,"items":[{"id":35980,"from_id":-30159897,"owner_id":-30159897,"date":1594971720,"marked_as_ads":0,"post_type":"post","text":"Знаем, отдыхать🌴 — не работать, но полностью отвлечься от рабочих дел тоже нужно уметь: найти силы не проверять рабочую почту, не хвататься за задачи, на которые в будни не оставалось времени, не нервничать «как они там без меня». \\n\\nПоделитесь, что помогает вам переключиться и перезагрузиться за выходные?","is_pinned":1,"attachments":[{"type":"photo","photo":{"album_id":-7,"date":1594896138,"id":457244356,"owner_id":-30159897,"has_tags":false,"access_key":"f734b166b060179414","post_id":35980,"sizes":[{"height":87,"url":"https:\\/\\/sun1-95.userapi.com\\/kU6zk25x-6QNIZohT9rh-CXOQIEqbCTx3JF6GQ\\/lYft85AGNZs.jpg","type":"m","width":130},{"height":87,"url":"https:\\/\\/sun1-95.userapi.com\\/kU6zk25x-6QNIZohT9rh-CXOQIEqbCTx3JF6GQ\\/lYft85AGNZs.jpg","type":"o","width":130},{"height":133,"url":"https:\\/\\/sun1-22.userapi.com\\/neYKzMUoSPc

In [57]:
r.json()

{'response': {'count': 7095,
  'items': [{'id': 35980,
    'from_id': -30159897,
    'owner_id': -30159897,
    'date': 1594971720,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Знаем, отдыхать🌴 — не работать, но полностью отвлечься от рабочих дел тоже нужно уметь: найти силы не проверять рабочую почту, не хвататься за задачи, на которые в будни не оставалось времени, не нервничать «как они там без меня». \n\nПоделитесь, что помогает вам переключиться и перезагрузиться за выходные?',
    'is_pinned': 1,
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -7,
       'date': 1594896138,
       'id': 457244356,
       'owner_id': -30159897,
       'has_tags': False,
       'access_key': 'f734b166b060179414',
       'post_id': 35980,
       'sizes': [{'height': 87,
         'url': 'https://sun1-95.userapi.com/kU6zk25x-6QNIZohT9rh-CXOQIEqbCTx3JF6GQ/lYft85AGNZs.jpg',
         'type': 'm',
         'width': 130},
        {'height': 87,
         'url': 'https://su

In [63]:
pd.DataFrame(r.json()['response']['items'])

,id,from_id,owner_id,date,marked_as_ads,post_type,text,is_pinned,attachments,post_source,comments,likes,reposts,views,edited
0,35980,-30159897,-30159897,1594971720,0,post,"Знаем, отдыхать🌴 — не работать, но полностью о...",1.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 13, 'can_post': 1, 'groups_can_post'...","{'count': 12, 'user_likes': 0, 'can_like': 1, ...","{'count': 3, 'user_reposted': 0}",{'count': 3813},NaN
1,36000,-30159897,-30159897,1595083320,0,post,"Выступать онлайн — это не врождённый талант, а...",NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 5, 'user_likes': 0, 'can_like': 1, '...","{'count': 1, 'user_reposted': 0}",{'count': 1641},NaN
2,35999,-30159897,-30159897,1595061720,0,post,Мероприятия нашей афиши по-прежнему онлайн — з...,NaN,"[{'type': 'link', 'link': {'url': 'https://m.v...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 3, 'user_likes': 0, 'can_like': 1, '...","{'count': 2, 'user_reposted': 0}",{'count': 1937},NaN
3,35996,-30159897,-30159897,1594996920,0,post,Ху ис он дьюти тудей?👩‍🏫 Подтягиваем деловой а...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 8, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{'count': 2968},NaN
4,35978,-30159897,-30159897,1594917720,0,post,У контент-плана есть масса преимуществ: он пом...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 46, 'user_likes': 0, 'can_like': 1, ...","{'count': 8, 'user_reposted': 0}",{'count': 5625},NaN
5,35962,-30159897,-30159897,1594884420,0,post,Про тайм-менеджмент написано много книг и курс...,NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 15, 'user_likes': 0, 'can_like': 1, ...","{'count': 1, 'user_reposted': 0}",{'count': 5448},1.594886e+09
6,35959,-30159897,-30159897,1594823220,0,post,Как стать программистом? Разберёмся на бесплат...,NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 7, 'user_likes': 0, 'can_like': 1, '...","{'count': 2, 'user_reposted': 0}",{'count': 3104},NaN
7,35950,-30159897,-30159897,1594798020,0,post,Пишите свои варианты ответов в комментарии. А ...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 8, 'can_post': 1, 'groups_can_post':...","{'count': 9, 'user_likes': 0, 'can_like': 1, '...","{'count': 0, 'user_reposted': 0}",{'count': 5272},NaN
8,35949,-30159897,-30159897,1594739820,0,post,Отказываться бывает сложно — мы боимся выгляде...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 31, 'user_likes': 0, 'can_like': 1, ...","{'count': 7, 'user_reposted': 0}",{'count': 5158},NaN
9,35946,-30159897,-30159897,1594712820,0,post,"Что делать, если работа не приносит ни радости...",NaN,"[{'type': 'link', 'link': {'url': 'http://neto...",{'type': 'vk'},"{'count': 0, 'can_post': 1, 'groups_can_post':...","{'count': 32, 'user_likes': 0, 'can_like': 1, ...","{'count': 7, 'user_reposted': 0}",{'count': 4338},NaN
